<a href="https://colab.research.google.com/github/ShravankumarMR/BigData-Spark/blob/main/UberAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Uber month wise trips for all base numbers').getOrCreate()
sc = spark.sparkContext

# Read data from txt.
The data is provided in below structure:

1)base no
2)dt
3)no of active vehicles
4)no of trips

In [7]:
inp = sc.textFile("/content/Input/uber_data")

print(" Total records : ", inp.count() , " \n\n Sample records: \n")
for i in inp.take(5):
  print(i)

 Total records :  355  

 Sample records: 

dispatching_base_number,date,active_vehicles,trips
B02512,1/1/2015,190,1132
B02765,1/1/2015,225,1765
B02764,1/1/2015,3427,29421
B02682,1/1/2015,945,7679


In [9]:
print("Parallelism supported: ", sc.defaultParallelism)
print("Total partitions: ", inp.getNumPartitions())

Parallelism supported:  2
Total partitions:  2


In [10]:
# Data has header so extract rdd without header
header = inp.first()
data = inp.filter(lambda x: x!=header)

print("Total records after removing header: ", data.count())

Total records after removing header:  354


In [29]:
# Get date field , trips
rdd1 = data.map(lambda a : (a.split(',')[0],a.split(',')[1] , a.split(',')[3]))

for i in rdd1.take(5):
  print(i)


('B02512', '1/1/2015', '1132')
('B02765', '1/1/2015', '1765')
('B02764', '1/1/2015', '29421')
('B02682', '1/1/2015', '7679')
('B02617', '1/1/2015', '9537')


In [20]:

todays_date = "10/05/2025"
format_date = "%m/%d/%Y"
print(datetime.strptime(todays_date, format_date).strftime("%B"))

October


In [32]:
# Extract month from date
from datetime import datetime
rdd2 = rdd1.map(lambda a : (a[0],datetime.strptime(a[1], format_date).strftime("%B"),a[2]))

for i in rdd2.take(5):
  print(i)

('B02512', 'January', '1132')
('B02765', 'January', '1765')
('B02764', 'January', '29421')
('B02682', 'January', '7679')
('B02617', 'January', '9537')


In [24]:
tripsPerMonth = rdd2.reduceByKey(lambda a,b : int(a)+int(b))

print(" Total trips per month : \n")
for i in tripsPerMonth.take(5):
  print(i)

 Total trips per month : 

('January', 1908649)
('February', 2221581)


In [34]:
kvRdd = rdd2.map(lambda a : (a[0] +' ' +  a[1] ,a[2]))
tripsPerMonthPerBase = kvRdd.reduceByKey(lambda a,b : int(a)+int(b))

print(" Total trips per month per base: \n")
for i in tripsPerMonthPerBase.take(5):
  print(i)

 Total trips per month per base: 

('B02765 January', 56287)
('B02764 January', 915976)
('B02682 January', 295941)
('B02617 January', 345988)
('B02598 January', 251658)


In [25]:
print(" Partitions for the output :" , tripsPerMonth.getNumPartitions())

 Partitions for the output : 2


In [26]:
output = tripsPerMonth.coalesce(1)

# Save data to output file

output.saveAsTextFile("/content/Output")


In [37]:
output_withbase = tripsPerMonthPerBase.coalesce(1)
# Save data to output file

output_withbase.saveAsTextFile("/content/Output1")

In [27]:
# print saved output for demo

rdd3 = sc.textFile("/content/Output/part-00000")

for i in rdd3.take(5):
  print(i)

('January', 1908649)
('February', 2221581)


In [39]:
# print saved output for demo

rdd4 = sc.textFile("/content/Output1/part-00000")

for i in rdd4.take(10):
  print(i)

('B02765 January', 56287)
('B02764 January', 915976)
('B02682 January', 295941)
('B02617 January', 345988)
('B02598 January', 251658)
('B02598 February', 289133)
('B02512 January', 42799)
('B02682 February', 366568)
('B02512 February', 50987)
('B02765 February', 137383)
